In [27]:
import pandas as pd
import numpy as np
import json
from haralyzer import HarParser
import browsermobproxy as mob
from selenium import webdriver
import time

In [30]:
mob_proxy_path = "C:/Program Files/browsermob-proxy-2.1.4/bin/browsermob-proxy.bat"
url = "https://soundcloud.com/search/people?q=*"

s = mob.Server(mob_proxy_path)
s.start()

proxy = s.create_proxy()
#profile.set_proxy(proxy.selenium_proxy())
driver = webdriver.Chrome()

proxy.new_har("soundcloud.com.har", options={'captureHeaders': True, 'captureContent':True})
driver.get(url)
time.sleep(10)

print(proxy.har)

s.stop()
driver.quit()

{'log': {'version': '1.2', 'creator': {'name': 'BrowserMob Proxy', 'version': '2.1.4', 'comment': ''}, 'pages': [{'id': 'soundcloud.com.har', 'startedDateTime': '2022-06-16T22:20:02.052+01:00', 'title': 'soundcloud.com.har', 'pageTimings': {'comment': ''}, 'comment': ''}], 'entries': [], 'comment': ''}}


In [21]:
print(proxy.har["log"])

{'version': '1.2', 'creator': {'name': 'BrowserMob Proxy', 'version': '2.1.4', 'comment': ''}, 'pages': [{'id': 'soundcloud.com.har', 'startedDateTime': '2022-06-16T21:57:10.643+01:00', 'title': 'soundcloud.com.har', 'pageTimings': {'comment': ''}, 'comment': ''}], 'entries': [], 'comment': ''}


The file is read and dictioary is created. I need to create a cell that can automate the HAR extraction process.

In [22]:
with open("soundcloud.com.har", "r", encoding="utf8") as f:
    har_parser = HarParser(json.loads(f.read()))

data = har_parser.har_data
#print(data["entries"][80]["response"]["content"]["text"])
#print(data["entries"][1]["response"]["content"].keys())

To access the description(artist bio) string slicing is used.

In [57]:
#sliced string example.

start_d = '"description":'
end_d = ',"followers_count"'

s =  data["entries"][48]["response"]["content"]["text"]
desc = s[s.find(start_d) + len(start_d): s.find(end_d)]

desc

'"After the release of their 1st LP ‘Sound of Freedom’ Paradisia quickly earned a reputation for their trademark harmonies and the unique delicacy of the harp featuring a critically acclaimed cover of Springsteen’s ‘Dancing In The Dark’ praised by the original song’s producer himself Jon Landau, accumulating over 4 millions streams, and for their ever relevant Feminist single ‘Warpaint’. Paradisia followed up with a European headline tour, and supported the likes of Michael Kiwanuka, Laura Mvula and the legendary Mr Bryan Ferry amongst others.\\n\\nThe girls have come home to regain their independence, after leaving their label and management they are about to embark on putting out an entirely self produced release. Paradisia are a self made DIY wonder. From the visuals, songwriting to the production, they are a multi dimensional creative force that won’t take no for an answer."'

In [58]:

#dictionary for containment of page names and corresponding description strings is initialised.


def extract_info(data): 
    data_dict = {"page_name" :[], "description":[]}

    start_d = '"description":'
    end_d = ',"followers_count"'

    start_n = '"full_name":'
    end_n = ',"groups_count"'
    for i in range (0, len(data["entries"])):
        if "encoding" in data["entries"][i]["response"]["content"].keys():
            
            continue
        elif "text" in data["entries"][i]["response"]["content"].keys():
            s = data["entries"][i]["response"]["content"]["text"]
            desc = s[s.find(start_d) + len(start_d): s.find(end_d)]
            name = s[s.find(start_n) + len(start_n): s.find(end_n)]
            data_dict["description"].append(desc)
            data_dict["page_name"].append(name)
        else:
            continue
    return pd.DataFrame(data_dict)

df = extract_info(data)
df.replace('""', np.nan, inplace=True)
df.replace("null", np.nan, inplace=True)
df.replace("", np.nan, inplace=True)
df.dropna(inplace=True)
df


,page_name,description
0,"""SiriusXM Sports""","""Official SiriusXM Sports SoundCloud Page, fea..."
1,"""Rádio Auri Verde""","""Às vésperas de completar 60 anos, a Rádio Aur..."
6,"""Dwitasari Dwita""","""Contact & Buzzer : @Aanstory 081286764725 / 0..."
11,"""Ola Englund""","""BUY MY SHIT AT www.fearedband.com"""
13,"""Jad Abumrad""","""Radiolab is a show about curiosity. Hosts Jad..."
15,"""Sony Music Gospel""","""Soundcloud oficial da Sony Music Gospel, ouça..."
16,"""triple j Hack""","""Shoving the J into journalism, Hack covers cu..."
17,"""Check out the action on the United_ (http://w...","""Join us at <a href=http://www.soundcloud.com/..."
18,"""1938 News""","""1938 News."""
19,"""Khuntan Wich""","""กาเลนะ ธัมมัสสะวะนัง เอตัมมังคะละมุตตะมัง.\n\..."
